In [1]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Load datasets
mentors = pd.read_csv("Mentor.csv")  # Columns: Name, Sector 1, Sector 2, Sector 3, Index
startups = pd.read_csv("Startups.csv")  # Columns: Name, Sector, Index

# Initialize time slots
start_time = datetime.strptime("11:00 AM", "%I:%M %p")
end_time = datetime.strptime("2:00 PM", "%I:%M %p")
time_slots = []
slot_duration = timedelta(minutes=15)
gap_duration = timedelta(minutes=5)

while start_time + slot_duration <= end_time:
    time_slots.append(start_time.strftime("%I:%M %p"))
    start_time += slot_duration + gap_duration

# Helper function to assign startups to mentors iteratively
def assign_startups_iteratively(mentors, startups, time_slots):
    mentor_schedule = {mentor["Name"]: [] for _, mentor in mentors.iterrows()}
    startup_counts = {startup["Name"]: 0 for _, startup in startups.iterrows()}
    assigned_startups = set()

    # Prepare preferences for each mentor
    mentor_preferences = {
        mentor["Name"]: [mentor["Sector 1"], mentor["Sector 2"], mentor["Sector 3"]]
        for _, mentor in mentors.iterrows()
    }
    weights = [60, 24, 16]

    # Round-robin assignment
    slot_index = 0
    while slot_index < len(time_slots):
        for _, mentor in mentors.iterrows():
            mentor_name = mentor["Name"]
            preferences = mentor_preferences[mentor_name]

            # Filter startups based on preferences and not exceeding repetition
            available_startups = startups[
                (startups["Name"].isin(startup_counts.keys())) &
                (startups["Sector"].isin(preferences)) &
                (startups["Name"].map(startup_counts) < 4)
            ]

            if available_startups.empty:
                continue

            # Select a startup based on weighted preferences
            chosen_sector = random.choices(preferences, weights=weights, k=1)[0]
            sector_startups = available_startups[available_startups["Sector"] == chosen_sector]

            if sector_startups.empty:
                continue

            startup = sector_startups.sample(1).iloc[0]
            startup_name = startup["Name"]

            # Assign the startup to the mentor for the current time slot
            if slot_index < len(time_slots):
                mentor_schedule[mentor_name].append({
                    "Name": startup_name,
                    "Sector": startup["Sector"],
                    "Time Slot": time_slots[slot_index]
                })
                slot_index += 1
                startup_counts[startup_name] += 1
                assigned_startups.add(startup_name)

    return mentor_schedule, startup_counts

# Assign startups to mentors
schedule, startup_counts = assign_startups_iteratively(mentors, startups, time_slots)

# Display the schedule
print("Mentoring Schedule:")
for _, mentor in mentors.iterrows():
    print(f"\nMentor: {mentor['Name']}")
    print(f"Preferences: {mentor['Sector 1']}, {mentor['Sector 2']}, {mentor['Sector 3']}")
    print("Scheduled Startups:")
    for session in schedule[mentor["Name"]]:
        print(f"  - {session['Name']} ({session['Sector']}) at {session['Time Slot']}")

# Display startup counts
print("\nStartup Session Counts:")
for startup, count in startup_counts.items():
    print(f"{startup}: {count} sessions")


Mentoring Schedule:

Mentor: Mentor 1
Preferences: B2B Software, Marketplace, Media/Media Tech
Scheduled Startups:
  - Startup 123 (B2B Software) at 11:00 AM

Mentor: Mentor 2
Preferences: Consumer goods and services, HR Tec, Government
Scheduled Startups:
  - Startup 6 (Consumer goods and services) at 11:20 AM

Mentor: Mentor 3
Preferences: Industrial, Deep Tech, Blockchain
Scheduled Startups:
  - Startup 1  (Blockchain) at 11:40 AM

Mentor: Mentor 4
Preferences: Web3, B2B Software, Government
Scheduled Startups:
  - Startup 83 (Web3) at 12:00 PM

Mentor: Mentor 5
Preferences: Government, Healthcare, Healthcare
Scheduled Startups:
  - Startup 148 (Healthcare) at 12:20 PM

Mentor: Mentor 6
Preferences: Blockchain, Healthcare, Industrial
Scheduled Startups:
  - Startup 108 (Blockchain) at 12:40 PM

Mentor: Mentor 7
Preferences: Web3, Fintech, Fintech
Scheduled Startups:
  - Startup 83 (Web3) at 01:00 PM

Mentor: Mentor 8
Preferences: Energy And Environment, Blockchain, Consumer goods an